In [1]:
import asyncio
import os
from pydub import AudioSegment
from shazamio import Shazam
import yt_dlp
import nest_asyncio
from moviepy import VideoFileClip

nest_asyncio.apply()

async def download_audio(youtube_url: str, output_path: str, cookies_path: str):
    ydl_opts = {
        'format': 'bestvideo+bestaudio/best',
        'outtmpl': output_path,
        'noplaylist': True,         # Ensure only the single video is downloaded
        'overwrites': True,         # Overwrite the file if it already exists
        'cookies': cookies_path,    # Use cookies file
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([youtube_url])

async def recognize_track(file_path: str, segment_duration: int = 8):
    shazam = Shazam()
    audio = AudioSegment.from_file(file_path)
    duration = len(audio) // 1000  # Convert to seconds
    results = []
    i = 0
    for start in range(0, duration, segment_duration):
        end = min(start + segment_duration, duration)
        segment = audio[start * 1000:end * 1000]
        file_path = f"../data/temp_segment{i}.mp3"
        segment.export(file_path, format="mp3")
        out = await shazam.recognize(file_path)
        os.remove(file_path)
        # print(out)
        if 'track' in out:
            track = out['track']
            song_title = track.get('title', 'Unknown Title')
            artist_name = track.get('subtitle', 'Unknown Artist')
            album_name = track.get('sections', [{}])[0].get('metadata', [{}])[0].get('text', 'Unknown Album')
            genre = track.get('genres', {}).get('primary', 'Unknown Genre')
            # print(f"Segment {i}: {start}-{end}, Song: {song_title}, Artist: {artist_name}, Album: {album_name}, Genre: {genre}")
            results.append({
                'segment': i,
                'start': start,
                'end': end,
                'song': song_title,
                'artist': artist_name,
                'album': album_name,
                'genre': genre
            })
        else:
            # print("Could not recognize the track.")
            results.append({
                'segment': i,
                'start': start,
                'end': end,
                'song': None,
                'artist': None,
                'album': None,
                'genre': None
            })
        i += 1
    return results

def convert_mp4_to_mp3(mp4_file, mp3_file):
    video_clip = VideoFileClip(mp4_file)
    audio_clip = video_clip.audio
    audio_clip.write_audiofile(mp3_file)
    audio_clip.close()
    video_clip.close()

async def main():
    segment_duration = 5                                            # seconds
    cookies_file = '../data/cookies.txt'                            # Path to your cookies file
    # youtube_url = 'https://www.youtube.com/shorts/lTqjbKZZ9vk'      # hard
    # youtube_url = 'https://www.youtube.com/shorts/IykG5LsziGw'      # easy
    # youtube_url = 'https://www.youtube.com/watch?v=7vglAmBc9tY'     # detroit lion
    # audio_file = '../data/video.webm'
    # await download_audio(youtube_url, audio_file, cookies_file)

    # audio_file = '../data/harder.mp3'

    mp4_file = '../data/video_2_songs_easy.mp4'
    audio_file = '../data/video_audio.mp3'
    convert_mp4_to_mp3(mp4_file, audio_file)

    result = await recognize_track(audio_file, segment_duration)
    print(result)

await main()


{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'mp42', 'minor_version': '1', 'compatible_brands': 'isommp41mp42', 'creation_time': '2025-02-02T22:36:00.000000Z'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'audio', 'language': 'eng', 'default': True, 'fps': 48000, 'bitrate': 107, 'metadata': {'Metadata': '', 'creation_time': '2025-02-02T22:36:00.000000Z', 'handler_name': 'Core Media Audio', 'vendor_id': '[0][0][0][0]'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'video', 'language': None, 'default': True, 'size': [1920, 1080], 'bitrate': 18753, 'fps': 59.94, 'codec_name': 'h264', 'profile': '(High)', 'metadata': {'Metadata': '', 'creation_time': '2025-02-02T22:36:00.000000Z', 'handler_name': 'Core Media Video', 'vendor_id': '[0][0][0][0]'}}], 'input_number': 0}], 'duration': 48.82, 'bitrate': 18894, 'start': 0.0, 'default_audio_input_number': 0, 'default_audio_stream_number': 0, 'audio_fps': 48000, 'audio_bitrate'

MoviePy - Done.
[{'segment': 0, 'start': 0, 'end': 5, 'song': 'Sugar', 'artist': 'Maroon 5', 'album': 'V (Deluxe)', 'genre': 'Pop'}, {'segment': 1, 'start': 5, 'end': 10, 'song': 'Sugar', 'artist': 'Maroon 5', 'album': 'V (Deluxe)', 'genre': 'Pop'}, {'segment': 2, 'start': 10, 'end': 15, 'song': 'Sugar', 'artist': 'Maroon 5', 'album': 'V (Deluxe)', 'genre': 'Pop'}, {'segment': 3, 'start': 15, 'end': 20, 'song': 'What Makes You Beautiful', 'artist': 'One Direction', 'album': 'Up All Night', 'genre': 'Pop'}, {'segment': 4, 'start': 20, 'end': 25, 'song': 'What Makes You Beautiful', 'artist': 'One Direction', 'album': 'Up All Night', 'genre': 'Pop'}, {'segment': 5, 'start': 25, 'end': 30, 'song': 'What Makes You Beautiful', 'artist': 'One Direction', 'album': 'Up All Night', 'genre': 'Pop'}, {'segment': 6, 'start': 30, 'end': 35, 'song': 'What Makes You Beautiful', 'artist': 'One Direction', 'album': 'Up All Night', 'genre': 'Pop'}, {'segment': 7, 'start': 35, 'end': 40, 'song': 'What Mak